In [1]:
MLFLOW_TRACKING_URI = "../models/mlruns"
MLFLOW_RUN_ID = "a8fb8540585f471ab7bf110e5a28229a"

CLUSTERS_YAML_PATH = "../data/processed/features_skills_clusters_description.yaml"

In [3]:
import pandas as pd
import sys

sys.path.append("../scripts/")

import warnings
warnings.filterwarnings("ignore")
#from JobDeena import *
from JobPrediction import *

---
# Simulate additional skills
## 1. Calculate base probs

In [4]:
# Intialize model
model = JobPrediction(MLFLOW_TRACKING_URI, MLFLOW_RUN_ID, CLUSTERS_YAML_PATH)

In [5]:
available_skills = ["Unreal Engine", "Unity 3D"]#["Hadoop", "Python"]#,  "Git"]

In [6]:
base_predictions = model.predict_jobs_probabilities(available_skills)
base_predictions.sort_values(ascending=False)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    0.7s finished


Developer, game or graphics                      0.6000
Developer, front-end                             0.1720
Developer, mobile                                0.1560
Developer, full-stack                            0.1280
Developer, desktop or enterprise applications    0.1260
Scientist                                        0.0888
Academic researcher                              0.0880
Engineer, data                                   0.0680
System administrator                             0.0600
Developer, back-end                              0.0580
Data or business analyst                         0.0480
Developer, QA or test                            0.0352
Data scientist or machine learning specialist    0.0280
Developer, embedded applications or devices      0.0200
Database administrator                           0.0040
DevOps specialist                                0.0000
dtype: float64

## 2. Calculate Probs for new skills:

In [7]:
# Get possible new skills
all_skills = pd.Series(model.get_all_skills())
new_skills = all_skills[~all_skills.isin(available_skills)].copy()

In [8]:
new_skills

0                 APL
1            Assembly
2          Bash/Shell
3                   C
4                  C#
            ...      
142    skills_group_5
143    skills_group_6
144    skills_group_7
145    skills_group_8
146    skills_group_9
Length: 145, dtype: object

In [9]:
simulated_results = []
for skill in new_skills:
    additional_skills_prob = model.predict_jobs_probabilities([skill] + available_skills)
    additional_skill_uplift = (additional_skills_prob - base_predictions) / base_predictions
    additional_skill_uplift.name = skill
    simulated_results.append(additional_skill_uplift)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      |

In [10]:
simulated_results = pd.DataFrame(simulated_results)
simulated_results

,Academic researcher,Data or business analyst,Data scientist or machine learning specialist,Database administrator,DevOps specialist,"Developer, QA or test","Developer, back-end","Developer, desktop or enterprise applications","Developer, embedded applications or devices","Developer, front-end","Developer, full-stack","Developer, game or graphics","Developer, mobile","Engineer, data",Scientist,System administrator
APL,0.000000,0.000000,0.285714,0.000000,NaN,0.000000,0.068966,0.063492,0.000000,0.023256,-0.09375,-0.013333,0.000000,0.235294,-0.090090,0.400000
Assembly,-0.181818,-0.666667,-0.285714,2.000000,inf,0.530303,0.758621,0.404762,2.400000,-0.302326,-0.31250,-0.106000,0.128205,-0.352941,-0.105856,0.083333
Bash/Shell,0.378788,-0.250000,0.285714,6.333333,inf,-0.002165,2.091954,0.401361,1.666667,0.046512,-0.09375,-0.446667,0.333333,-0.764706,0.840626,0.179365
C,-0.500000,-1.000000,-0.714286,1.000000,NaN,-0.492424,0.310345,0.047619,4.400000,-0.441860,-0.46875,0.106667,0.333333,-0.411765,-0.540541,-0.266667
C#,-0.409091,-0.416667,-0.857143,1.000000,NaN,-0.659091,-0.793103,-0.714286,-0.800000,-0.186047,-0.78125,0.433333,-0.205128,0.764706,-0.729730,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
skills_group_5,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
skills_group_6,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
skills_group_7,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
skills_group_8,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000


## Recommend

In [11]:
#target_job = "Data scientist or machine learning specialist"
target_job = "Engineer, data"

In [12]:
target_results = simulated_results[target_job].sort_values(ascending=False)
target_results

R                        1.529412
Julia                    1.352941
NumPy                    1.058824
Matlab                   1.058824
Pandas                   0.963235
                           ...   
Android Studio          -0.647059
Google Cloud Platform   -0.705882
JavaScript              -0.705882
Firebase                -0.705882
Bash/Shell              -0.764706
Name: Engineer, data, Length: 145, dtype: float64

In [13]:
threshold = 0
recommendations = target_results[target_results > threshold].index.tolist()

In [14]:
recommendations

['R',
 'Julia',
 'NumPy',
 'Matlab',
 'Pandas',
 'C#',
 '.NET Framework',
 '.NET Core / .NET 5',
 'DynamoDB',
 'COBOL',
 'IBM DB2',
 'Flow',
 'Xamarin',
 'APL',
 'Chef',
 'Crystal',
 'PowerShell',
 'Microsoft SQL Server',
 'Puppet',
 'Hadoop',
 'PHPStorm',
 'Apache Spark',
 'React.js',
 'Python',
 'ASP.NET Core ',
 'Delphi',
 'FastAPI',
 'SQL',
 'Pulumi',
 'Atom',
 'Neovim',
 'RStudio',
 'Terraform',
 'TextMate']

In [15]:
print(f"Your current skills: {str(available_skills)}")
print(f"Your target job: {str(target_job)}")
print(f"You might also consider learning: {recommendations}")

Your current skills: ['Unreal Engine', 'Unity 3D']
Your target job: Engineer, data
You might also consider learning: ['R', 'Julia', 'NumPy', 'Matlab', 'Pandas', 'C#', '.NET Framework', '.NET Core / .NET 5', 'DynamoDB', 'COBOL', 'IBM DB2', 'Flow', 'Xamarin', 'APL', 'Chef', 'Crystal', 'PowerShell', 'Microsoft SQL Server', 'Puppet', 'Hadoop', 'PHPStorm', 'Apache Spark', 'React.js', 'Python', 'ASP.NET Core ', 'Delphi', 'FastAPI', 'SQL', 'Pulumi', 'Atom', 'Neovim', 'RStudio', 'Terraform', 'TextMate']
